In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta

In [2]:
dirProc = 'data/processed/'

arrestsDirty = pd.read_csv((dirProc + 'arrestdata-all.csv'), index_col='Unnamed: 0')
homeGames = pd.read_csv((dirProc + 'iowa-home-games.csv'))

/home/user/.anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# helper function
def toDate(x):
    return datetime.strptime(x, '%Y-%m-%d')

# get set of dates
gameDates = homeGames['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d')).tolist()
homeGames['Date'] = homeGames['Date'].apply(lambda x: toDate(x))

In [4]:
# get game information 
def getLookup(x):
    #check if theres a day at all
    if (x['arrweekday']):
        
        # Calc day
        day = x['arrweekday']
        weekday = 1 if day in ['Friday', 'Saturday', 'Sunday'] else 0

        if (weekday == 1):
            if (day == 'Saturday'):
                lookup = toDate(x['arrdate'])
            elif (day == 'Friday'):
                lookup = toDate(x['arrdate']) + timedelta(days=1)
            else:
                lookup = toDate(x['arrdate']) - timedelta(days=1)  
            return (lookup)


In [7]:

def getGame(x):
    
    #check if theres a day at all
    if (x['arrweekday']):
        
        # Calc day
        day = x['arrweekday']
        weekday = 1 if day in ['Friday', 'Saturday', 'Sunday'] else 0

        if (weekday == 1):
            lookup = getLookup(x)
    
            gameWeekend = 1 if (lookup in gameDates) else 0
            return gameWeekend
        else:
            return 0
    # return 0 if theres no day
    else:
        return 0
    
def getDelta(x):
    #check if its a gameweekend
    if (x['gameWeekend'] == 1):
        lookup = getLookup(x)
        try:
            gameDelta = homeGames[homeGames['Date'] == lookup]['Delta'].iat[0]
        except:
            gameDelta = ''
        return gameDelta
    else:
        return False

def getTeam(x):
    #check if its a gameweekend
    if (x['gameWeekend'] == 1):
        lookup = getLookup(x)
        gameTeam = homeGames[homeGames['Date'] == lookup]['Visitor'].iat[0]
        return gameTeam
    
def getWin(x):
    #check if its a gameweekend
    if (x['gameWeekend'] == 1):
        lookup = getLookup(x)
        gameWin = homeGames[homeGames['Date'] == lookup]['Win'].iat[0]
        return gameWin

In [8]:
arrestsDirty['gameWeekend'] = arrestsDirty.apply(lambda x: getGame(x), axis=1)
arrestsDirty['gameDelta'] = arrestsDirty.apply(lambda x: getDelta(x), axis=1)
arrestsDirty['gameTeam'] = arrestsDirty.apply(lambda x: getTeam(x), axis=1)
arrestsDirty['gameWin'] = arrestsDirty.apply(lambda x: getWin(x), axis=1)

In [9]:
arrestsDirty.to_csv((dirProc + 'arrests-games-comb.csv'), index=False)